[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


In [4]:
import requests
import base64
from copy import deepcopy
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from textblob import TextBlob
from bs4 import BeautifulSoup as bsoup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
import re
import ast
import emoji

In [2]:
client_key = "get your client key from your twitter acct"
client_secret = "get your client key from your twitter acct" 

In [5]:
key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [6]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}

Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [7]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [8]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAEskIQEAAAAAqkugIbkkPuB3XUUBhHAYkbYX4ik%3DjxVjLVRb5O9OUZ95hy8Z3cDHaTkA4j7VJM0Shx3GNLq2W8SdOA'}

In [9]:
access_token = json_data['access_token']

# Get Tweets
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [10]:
max_tweets=1000 #CHANGED TO 200 FOR TESTING - MUST BE at least 1000 for analysis
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []

In [11]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

keywords = ["Packers","DaBears","Eagles", "Cowboys","Raiders", "ChiefsKingdom"]

for keyword in keywords:
    #parameters popular and extended tweet mode
    parameters = { 'q': keyword,
                        'result_type': 'mixed',
                    'count': tw_limit, 
                     'tweet_mode': 'extended'}

    search_url = base_url+'1.1/search/tweets.json'

    response = requests.get(search_url, headers=search_headers, params=parameters)
    football_tweets = response.json()

    for status in football_tweets['statuses']:
        status["team"] = keyword
    tweets += football_tweets['statuses']
    ids = [tw['id'] for tw in football_tweets['statuses']]
    min_id = min(ids)
    tw_ids = []
    tw_ids+=ids

    for  i in range(max_tweets//tw_limit -1):
        parameters = { 'q': keyword,
                        'result_type': 'mixed',
                    'count': tw_limit,
                    'tweet_mode': 'extended',
                    'max_id': min(tw_ids)-1
                     }
        print("searching tweets with id  < {}".format(min_id))
        search_url = base_url+'1.1/search/tweets.json'
        response = requests.get(search_url, headers=search_headers, params=parameters)
        football_tweets = response.json()
        ids = [tw['id'] for tw in football_tweets['statuses']]
        tw_ids+=ids
        print(i, len(football_tweets['statuses']))
        min_id = min(ids)
        for status in football_tweets['statuses']:
            status["team"] = keyword
        tweets += football_tweets['statuses']    

searching tweets with id  < 1329443531694481411
0 100
searching tweets with id  < 1329435997503971329
1 100
searching tweets with id  < 1329427456558829568
2 100
searching tweets with id  < 1329420217181745160
3 100
searching tweets with id  < 1329407906836590594
4 100
searching tweets with id  < 1329398277880619008
5 100
searching tweets with id  < 1329380324388716544
6 100
searching tweets with id  < 1329349959305445376
7 100
searching tweets with id  < 1329329823873986571
8 100
searching tweets with id  < 1329502616842276865
0 100
searching tweets with id  < 1329451542794170371
1 100
searching tweets with id  < 1329306243983429633
2 100
searching tweets with id  < 1329245864729202693
3 100
searching tweets with id  < 1329197314142035969
4 100
searching tweets with id  < 1329147678471712769
5 100
searching tweets with id  < 1329097265768767501
6 100
searching tweets with id  < 1328959096545030148
7 100
searching tweets with id  < 1328851179598647303
8 100
searching tweets with id  < 

As per twitter api documentation we can use

max_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

In [12]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 6000


# Let's print some tweets

In [13]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [14]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Thu Nov 19 15:16:52 +0000 2020',
  'display_text_range': [0, 202],
  'entities': {'hashtags': [],
               'media': [{'display_url': 'pic.twitter.com/xlo5NGg6iD',
                          'expanded_url': 'https://twitter.com/briebriejoy/status/1329443531694481411/photo/1',
                          'id': 1329443527311450121,
                          'id_str': '1329443527311450121',
                          'indices': [203, 226],
                          'media_url': 'http://pbs.twimg.com/media/EnMh2CpXYAkQrUs.jpg',
                          'media_url_https': 'https://pbs.twimg.com/media/EnMh2CpXYAkQrUs.jpg',
                          'sizes': {'large': {'h': 444,
                                              'resize': 'fit',
                                              'w': 1125},
                                    'medium': {'h': 444,
                                               'resize': 'fit',
           

In [15]:
for status in tweets[0:max_tweet_to_print]:
    print(status['full_text'] + '\n')

Thinking about how many Iowa meat packers voted for Bernie who would’ve given them ownership stake in their business, greater control over their conditions, and Medicare for All. https://t.co/wef0Do6kyu https://t.co/xlo5NGg6iD

Packers' WR Davante Adams did not practice today due to an ankle injury.

#Packers WR Davante Adams is officially off the injury report and will play Sunday against the #Colts, while WR Allen Lazard (core) is listed as questionable.

@marcellemroy Highlights are the #ALR &amp; BC Food Security, including the KPU Farm School on the Garden City Lands. Then #RichmondBC Garry Point &amp; Terra Nova Parks, Trails, Gulf of Georgia Cannery &amp; Britannia Shipyard. On the down side loss of BC Packers Cannery for a seafood market &amp; #SiteC Dam



# Pandas DataFrame for storing tweet feed


# Printing keys in tweets

In [16]:
print(tweets[1].keys())
print(len(tweets[1].keys()))

dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang', 'team'])
26


In [17]:
import pandas as pd

In [18]:
def tweet_generator(tweets):
    for tweet in tweets:
        modified_tweet= {}
        
        #Write your code here

        modified_tweet = deepcopy(tweet)
        try:
            modified_tweet["user_id"] = modified_tweet["user"]["id"]
            modified_tweet.pop("user", None)
        except KeyError:
            pass
        try:
            modified_tweet["retweeted_status_id"] = modified_tweet["retweeted_status"]["id"] 
            modified_tweet.pop("retweeted_status", None)
        except KeyError:
            pass
        yield modified_tweet       

In [19]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'created_at': 'Thu Nov 19 15:16:52 +0000 2020',
  'id': 1329443531694481411,
  'id_str': '1329443531694481411',
  'full_text': 'Thinking about how many Iowa meat packers voted for Bernie who would’ve given them ownership stake in their business, greater control over their conditions, and Medicare for All. https://t.co/wef0Do6kyu https://t.co/xlo5NGg6iD',
  'truncated': False,
  'display_text_range': [0, 202],
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [],
   'urls': [{'url': 'https://t.co/wef0Do6kyu',
     'expanded_url': 'https://twitter.com/dmregister/status/1329431739475832835',
     'display_url': 'twitter.com/dmregister/sta…',
     'indices': [179, 202]}],
   'media': [{'id': 1329443527311450121,
     'id_str': '1329443527311450121',
     'indices': [203, 226],
     'media_url': 'http://pbs.twimg.com/media/EnMh2CpXYAkQrUs.jpg',
     'media_url_https': 'https://pbs.twimg.com/media/EnMh2CpXYAkQrUs.jpg',
     'url': 'https://t.co/xlo5NGg6iD',
     'display

In [20]:
football_feed_df = pd.DataFrame(tweet_generator(tweets))

# Tweet Cleaning

In [22]:
print("Rows with either null text or teams: {}".format(football_feed_df[["full_text", "team"]].isnull().any(axis = 1).sum())) 

Rows with either null text or teams: 0


In [23]:
def NoneToNan(elem):
    if elem == "None" or elem == None:
        return np.nan
    else:
        return elem

football_feed_df = football_feed_df.applymap(NoneToNan)

In [25]:
football_feed_df[["full_text","team"]].head()

,full_text,team
0,Thinking about how many Iowa meat packers vote...,Packers
1,Packers' WR Davante Adams did not practice tod...,Packers
2,#Packers WR Davante Adams is officially off th...,Packers
3,@marcellemroy Highlights are the #ALR &amp; BC...,Packers
4,RT @johnnyc1952: The Packers have Rodgers \n\n...,Packers


In [27]:
#football_feed_df.full_text.fillna(football_feed_df.text, inplace=True)
football_feed_df["full_text"] = football_feed_df["full_text"].apply(str)
football_feed_df[["full_text","team"]]

,full_text,team
0,Thinking about how many Iowa meat packers vote...,Packers
1,Packers' WR Davante Adams did not practice tod...,Packers
2,#Packers WR Davante Adams is officially off th...,Packers
3,@marcellemroy Highlights are the #ALR &amp; BC...,Packers
4,RT @johnnyc1952: The Packers have Rodgers \n\n...,Packers
...,...,...
5995,Hii #ChiefsKingdom https://t.co/yKiFwSYUZw,ChiefsKingdom
5996,RT @PFN365: The @BxMovement: @brettyarrisPFN w...,ChiefsKingdom
5997,RT @ArtbyCaitlyn: Want a print of my “Forever ...,ChiefsKingdom
5998,RT @Rollicking_R8R: #ChiefsKingdom big mad bec...,ChiefsKingdom


In [28]:
print("Rows with either null text or teams: {}".format(football_feed_df[["full_text", "team"]].isnull().any(axis = 1).sum())) 

Rows with either null text or teams: 0


In [29]:
hashtags = []
user_mentions = []

for row in football_feed_df.itertuples():
    row = row.entities
    try:
        hashtags_in_row = row["hashtags"]
        hashtags.append(hashtags_in_row)
    except:
        hashtags.append('')
    try:
        user_mentions_in_row = row["user_mentions"][0]["name"]
        user_mentions.append(user_mentions_in_row)
    except:
        user_mentions.append('')

In [30]:
football_feed_df["hashtags"] = hashtags
football_feed_df["user_mentions"] = user_mentions
football_feed_df

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,metadata,source,...,favorite_count,favorited,retweeted,possibly_sensitive,lang,team,user_id,retweeted_status_id,hashtags,user_mentions
0,Thu Nov 19 15:16:52 +0000 2020,1329443531694481411,1329443531694481411,Thinking about how many Iowa meat packers vote...,False,"[0, 202]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1329443527311450121, 'id_str...","{'result_type': 'popular', 'iso_language_code'...","<a href=""http://twitter.com/download/iphone"" r...",...,2744,False,False,False,en,Packers,1332630685,NaN,[],
1,Thu Nov 19 21:19:15 +0000 2020,1329534730115702787,1329534730115702787,Packers' WR Davante Adams did not practice tod...,False,"[0, 72]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'result_type': 'popular', 'iso_language_code'...","<a href=""https://mobile.twitter.com"" rel=""nofo...",...,3856,False,False,NaN,en,Packers,51263592,NaN,[],
2,Fri Nov 20 20:19:18 +0000 2020,1329882030855892998,1329882030855892998,#Packers WR Davante Adams is officially off th...,False,"[0, 158]","{'hashtags': [{'text': 'Packers', 'indices': [...",NaN,"{'result_type': 'popular', 'iso_language_code'...","<a href=""https://about.twitter.com/products/tw...",...,727,False,False,NaN,en,Packers,36089053,NaN,"[{'text': 'Packers', 'indices': [0, 8]}, {'tex...",
3,Sat Nov 21 08:07:18 +0000 2020,1330060203845410817,1330060203845410817,@marcellemroy Highlights are the #ALR &amp; BC...,False,"[14, 310]","{'hashtags': [{'text': 'ALR', 'indices': [33, ...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",...,0,False,False,NaN,en,Packers,1709275855,NaN,"[{'text': 'ALR', 'indices': [33, 37]}, {'text'...",Notwhatuthink
4,Sat Nov 21 08:06:45 +0000 2020,1330060066708459523,1330060066708459523,RT @johnnyc1952: The Packers have Rodgers \n\n...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,NaN,en,Packers,65300014,1.329859e+18,[],John E. C.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Thu Nov 19 02:59:43 +0000 2020,1329258021256634371,1329258021256634371,Hii #ChiefsKingdom https://t.co/yKiFwSYUZw,False,"[0, 18]","{'hashtags': [{'text': 'ChiefsKingdom', 'indic...",NaN,"{'iso_language_code': 'und', 'result_type': 'r...","<a href=""http://twitter.com/download/android"" ...",...,3,False,False,False,und,ChiefsKingdom,91913782,NaN,"[{'text': 'ChiefsKingdom', 'indices': [4, 18]}]",
5996,Thu Nov 19 02:58:38 +0000 2020,1329257748157132801,1329257748157132801,RT @PFN365: The @BxMovement: @brettyarrisPFN w...,False,"[0, 119]","{'hashtags': [{'text': 'ChiefsKingdom', 'indic...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,0,False,False,False,en,ChiefsKingdom,105942077,1.329235e+18,"[{'text': 'ChiefsKingdom', 'indices': [81, 95]}]",Pro Football Network
5997,Thu Nov 19 02:58:27 +0000 2020,1329257702300786689,1329257702300786689,RT @ArtbyCaitlyn: Want a print of my “Forever ...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,NaN,en,ChiefsKingdom,2838158866,1.329182e+18,[],Caitlyn Snelling
5998,Thu Nov 19 02:58:08 +0000 2020,1329257622831296519,1329257622831296519,RT @Rollicking_R8R: #ChiefsKingdom big mad bec...,False,"[0, 140]","{'hashtags': [{'text': 'ChiefsKingdom', 'indic...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,NaN,en,ChiefsKingdom,886057852120453120,1.329227e+18,"[{'text': 'ChiefsKingdom', 'indices': [20, 34]}]",🗣I'm here for this


In [31]:
def clean_tweet(text):  

    url = re.compile('https://.+')
    rt = re.compile("RT")
    user = re.compile('@\S+\s')
 
    #'(RT|@.*\s|https://.+)'

    text = url.sub("", text)
    text = rt.sub("", text)
    text = user.sub("", text)

    return text

football_feed_df["full_text"] = football_feed_df["full_text"].apply(clean_tweet)

In [32]:
def emoji_retriever(text):
      return [(char, emoji.demojize(char)) for char in text if char in emoji.UNICODE_EMOJI]

football_feed_df["emoji"] = football_feed_df["full_text"].apply(emoji_retriever)

In [33]:
contraction_patterns = [(r'won\'t', 'will not'), (r'can\'t', 'cannot'),
(r'haven\'t', 'have not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)\'re', '\g<1> are')]

class contraction_replacer(object):
    def __init__(self, contraction_patterns):
        self._contraction_regexes = [(re.compile(p), replaced_text) for p, replaced_text in contraction_patterns]

    def do_contraction_normalization(self, text):
        for contraction_regex, replaced_text in self._contraction_regexes:
            text = contraction_regex.sub(replaced_text, text)
        return text

def preprocess_text(text):
    
    contraction_patterns = [(r'won\'t', 'will not'), (r'can\'t', 'cannot'),
    (r'haven\'t', 'have not'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)\'re', '\g<1> are')]

    replacer = contraction_replacer(contraction_patterns)
    text = replacer.do_contraction_normalization(text)
    text = nltk.tokenize.word_tokenize(text)

    stop_words = set(stopwords.words('english')) 
    
    # lemmatization
    wnlemma = WordNetLemmatizer()
    lemma_text = []
    for word in text:
        if word in stop_words:
            continue
        lemma_text.append(wnlemma.lemmatize(word))
    lemma_text = " ".join(lemma_text)

    return lemma_text

football_feed_df["processed_text"] = football_feed_df["full_text"].apply(preprocess_text)

In [41]:
# DataFrame to csv

football_feed_df.to_csv('football_feed_df_final.csv')

In [35]:
def sentiment_analysis(text):
    text = " ".join(text["processed_text"])
    text = TextBlob(text)

    return text.sentiment.polarity

teams_df = football_feed_df[["processed_text", "team"]].groupby("team")
sentiments = teams_df.apply(sentiment_analysis)

# Team Sentiment Analysis

Below represents the relative polarity of each team from each team on Twitter.

In [36]:
sentiments

team
ChiefsKingdom    0.030595
Cowboys          0.096143
DaBears          0.106277
Eagles           0.086787
Packers          0.137727
Raiders          0.092446
dtype: float64